In [1]:
import numpy as np
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

train_features=np.load("../extracted_features/FS-change3/input_features_free_train.npy")
labels_stress_train=np.load("../extracted_features/FS-change3/label_free_train.npy")


test_features=np.load("../extracted_features/FS-change3/input_features_free_test.npy")
labels_stress_test=np.load("../extracted_features/FS-change3/label_free_test.npy")


val_features=np.load("../extracted_features/FS-change3/input_features_free_val.npy")
labels_stress_val=np.load("../extracted_features/FS-change3/label_free_val.npy")


In [2]:
print(labels_stress_train.shape,labels_stress_train)

(1121,) [0. 0. 0. ... 1. 1. 1.]


In [3]:
batch_size=20
train_data = Dataset(train_features, labels_stress_train)
test_data=Dataset(test_features,labels_stress_test)
val_data=Dataset(val_features,labels_stress_val)
train_data_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [4]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [5]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [6]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_dim = 8
        self.hidden_dim = 500
        self.L=250
        self.layer_dim = 1
        self.output_dim=1
        self.fc1 = nn.Sequential(
            nn.Linear(self.hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.attention = nn.Sequential(
            nn.Linear(self.hidden_dim, self.L),
            nn.Tanh(),
            nn.Linear(self.L, 1)
        )
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))

        A = self.attention(out) 
        A = torch.transpose(A, 2,1)  
        A = F.softmax(A, dim=2) 
#         print(A.shape)
        M = torch.matmul(A, out) .squeeze(1)
#         print(M.shape)
        out=self.fc1(M)
#         print(out.shape)
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [7]:
feature_size = 8
LR = 0.0001
epoch = 200
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
loss_func = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [8]:
def train(model,train_data_loader):
    truth=[]
    preds=[]
    model.train()
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))

def evalue(model,best_acc1,val_data_loader,modelname,it):
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()
    best_model="./saved_models/best_model_change_fs"
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=f1_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(it)+".pth")
        modelname.append(best_model+'_epoch_'+str(it)+".pth")
        print('Best validation accuracy1 ', best_acc1)
    return best_acc1
    

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
best_acc1=0
modelname=[]
for it in range(epoch+1):
    train(model,train_data_loader)
    best_acc1=evalue(model,best_acc1,val_data_loader,modelname,it)

EPOCH  0
Train : Loss: 0.0113, Train acc1 : 0.6120
Val : Loss: 0.0182, Val acc1 : 0.3654
Best validation accuracy1  0.36538461538461536
EPOCH  1
Train : Loss: 0.0105, Train acc1 : 0.6753
Val : Loss: 0.0170, Val acc1 : 0.3333
EPOCH  2
Train : Loss: 0.0103, Train acc1 : 0.6780
Val : Loss: 0.0168, Val acc1 : 0.1961
EPOCH  3
Train : Loss: 0.0101, Train acc1 : 0.6922
Val : Loss: 0.0178, Val acc1 : 0.3111
EPOCH  4
Train : Loss: 0.0099, Train acc1 : 0.6896
Val : Loss: 0.0177, Val acc1 : 0.3543
EPOCH  5
Train : Loss: 0.0097, Train acc1 : 0.7021
Val : Loss: 0.0176, Val acc1 : 0.3778
Best validation accuracy1  0.37777777777777777
EPOCH  6
Train : Loss: 0.0095, Train acc1 : 0.7074
Val : Loss: 0.0184, Val acc1 : 0.4062
Best validation accuracy1  0.40625
EPOCH  7
Train : Loss: 0.0094, Train acc1 : 0.7119
Val : Loss: 0.0185, Val acc1 : 0.3892
EPOCH  8
Train : Loss: 0.0098, Train acc1 : 0.7092
Val : Loss: 0.0184, Val acc1 : 0.4141
Best validation accuracy1  0.41414141414141414
EPOCH  9
Train : Loss: 

EPOCH  87
Train : Loss: 0.0016, Train acc1 : 0.9697
Val : Loss: 0.0249, Val acc1 : 0.4365
EPOCH  88
Train : Loss: 0.0031, Train acc1 : 0.9233
Val : Loss: 0.0252, Val acc1 : 0.4412
EPOCH  89
Train : Loss: 0.0017, Train acc1 : 0.9661
Val : Loss: 0.0239, Val acc1 : 0.4646
EPOCH  90
Train : Loss: 0.0024, Train acc1 : 0.9402
Val : Loss: 0.0247, Val acc1 : 0.4188
EPOCH  91
Train : Loss: 0.0015, Train acc1 : 0.9723
Val : Loss: 0.0252, Val acc1 : 0.4490
EPOCH  92
Train : Loss: 0.0020, Train acc1 : 0.9616
Val : Loss: 0.0273, Val acc1 : 0.4314
EPOCH  93
Train : Loss: 0.0019, Train acc1 : 0.9554
Val : Loss: 0.0251, Val acc1 : 0.4286
EPOCH  94
Train : Loss: 0.0017, Train acc1 : 0.9599
Val : Loss: 0.0260, Val acc1 : 0.4249
EPOCH  95
Train : Loss: 0.0012, Train acc1 : 0.9741
Val : Loss: 0.0252, Val acc1 : 0.4410
EPOCH  96
Train : Loss: 0.0020, Train acc1 : 0.9483
Val : Loss: 0.0253, Val acc1 : 0.4365
EPOCH  97
Train : Loss: 0.0012, Train acc1 : 0.9741
Val : Loss: 0.0248, Val acc1 : 0.4467
EPOCH  98


EPOCH  178
Train : Loss: 0.0003, Train acc1 : 0.9938
Val : Loss: 0.0271, Val acc1 : 0.4286
EPOCH  179
Train : Loss: 0.0003, Train acc1 : 0.9929
Val : Loss: 0.0266, Val acc1 : 0.4127
EPOCH  180
Train : Loss: 0.0014, Train acc1 : 0.9688
Val : Loss: 0.0251, Val acc1 : 0.4688
EPOCH  181
Train : Loss: 0.0010, Train acc1 : 0.9750
Val : Loss: 0.0270, Val acc1 : 0.4227
EPOCH  182
Train : Loss: 0.0009, Train acc1 : 0.9786
Val : Loss: 0.0261, Val acc1 : 0.4444
EPOCH  183
Train : Loss: 0.0005, Train acc1 : 0.9902
Val : Loss: 0.0272, Val acc1 : 0.4124
EPOCH  184
Train : Loss: 0.0004, Train acc1 : 0.9920
Val : Loss: 0.0271, Val acc1 : 0.4205
EPOCH  185
Train : Loss: 0.0004, Train acc1 : 0.9929
Val : Loss: 0.0278, Val acc1 : 0.4162
EPOCH  186
Train : Loss: 0.0016, Train acc1 : 0.9652
Val : Loss: 0.0283, Val acc1 : 0.4257
EPOCH  187
Train : Loss: 0.0011, Train acc1 : 0.9759
Val : Loss: 0.0277, Val acc1 : 0.4242
EPOCH  188
Train : Loss: 0.0008, Train acc1 : 0.9848
Val : Loss: 0.0269, Val acc1 : 0.4043

In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.46 0.53 0.44 0.68 0.48 0.28


In [11]:
# print(model)

In [12]:
for i in range(len(modelname)):
    modeltest=Classifier()
    best_state=torch.load(modelname[i])
    modeltest.load_state_dict(best_state)
    modeltest.cuda()
    modeltest.eval()
    truth=[]
    preds=[]
    for minibatch in test_data_loader:
                X_test, Y1_test  = minibatch
                X_test=X_test.cuda()
                Y1_test=Y1_test.cuda()
                output_test = modeltest(X_test.float())
                output_test=output_test.squeeze(1)
                prediction = torch.ge(output_test, 0.5).float()
                truth.extend(Y1_test.tolist())
                preds.extend(prediction.tolist())
    acc=accuracy_score(truth,preds)
    # print(truth,preds)
    tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
    f1score=f1_score(truth, preds)
    precision=precision_score(truth, preds)
    recall=recall_score(truth,preds)
    roc=roc_auc_score(truth,preds)
    specificity=tn/(tn+fp)

    print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))

0.49 0.50 0.45 0.56 0.50 0.44
0.36 0.16 0.19 0.13 0.34 0.54
0.40 0.29 0.31 0.27 0.39 0.51
0.40 0.33 0.33 0.33 0.39 0.45
0.38 0.23 0.26 0.21 0.36 0.51
0.39 0.25 0.28 0.22 0.37 0.52
0.42 0.38 0.37 0.40 0.42 0.43
0.46 0.53 0.43 0.67 0.48 0.28
0.46 0.53 0.44 0.68 0.48 0.28
